In [96]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [114]:
#Create Scraper Soup
sauce = requests.get("https://www.basketball-reference.com/leagues/NBA_2018_totals.html").text
soup = BeautifulSoup(sauce, "lxml")
del sauce
    
#Extract Table Content
db_data = [[data.text for data in record.find_all("td")] for record in soup.find_all("tr")]
   
#Extract Table Header
cols = [header.text for header in soup.find("tr").find_all("th")]

#Create DataFrame    
data = pd.DataFrame(db_data[1:], columns = cols[1:])
data.dropna(how="all", axis=0, inplace=True)



In [116]:
data.loc[:,"G":] = data.loc[:,"G":].apply(pd.to_numeric)

In [56]:
#Keep to reference split function
#name_split = lambda x: x["Player"].split("\\")[0]
#data["Player"] = data.apply(name_split, axis=1)
#del name_split

In [117]:
raw_stats = pd.concat([data["MP"], data["TRB"], data["AST"], data["STL"], data["BLK"], data["TOV"], data["PTS"]], axis=1)
per_game_label = ["MP/G", "TRB/G", "AST/G", "STL/G", "BLK/G", "TOV/G", "PTS/G"] 
raw_stats.columns = per_game_label
del per_game_label

In [118]:
games = data["G"]

def normalize (series1):
    return (series1/games)

In [119]:
raw_stats= raw_stats.apply(normalize)
raw_stats= raw_stats.round(2)
data = pd.concat([data, raw_stats], axis=1)
del raw_stats
del games

In [120]:
print(data.head(10))

              Player Pos Age   Tm   G  GS    MP   FG   FGA    FG%  ...    TOV  \
0       Alex Abrines  SG  24  OKC  75   8  1134  115   291  0.395  ...     25   
1         Quincy Acy  PF  27  BRK  70   8  1359  130   365  0.356  ...     60   
2       Steven Adams   C  24  OKC  76  76  2487  448   712  0.629  ...    128   
3        Bam Adebayo   C  20  MIA  69  19  1368  174   340  0.512  ...     66   
4      Arron Afflalo  SG  32  ORL  53   3   682   65   162  0.401  ...     21   
5       Cole Aldrich   C  29  MIN  21   0    49    5    15  0.333  ...      1   
6  LaMarcus Aldridge   C  32  SAS  75  75  2509  687  1347   0.51  ...    111   
7      Jarrett Allen   C  19  BRK  72  31  1441  234   397  0.589  ...     82   
8       Kadeem Allen  PG  25  BOS  18   1   107    6    22  0.273  ...      9   
9         Tony Allen  SF  36  NOP  22   0   273   44    91  0.484  ...     19   

    PF   PTS   MP/G TRB/G AST/G STL/G BLK/G TOV/G  PTS/G  
0  124   353  15.12  1.52  0.37  0.51  0.11  0.33

In [121]:
sauce = requests.get("https://www.basketball-reference.com/contracts/players.html").text
soup = BeautifulSoup(sauce, "lxml")
del sauce

#Import Data 
db_data = [[data.text for data in record.find_all("td")] for record in soup.find_all("tr")]

#Create Header
contract_header = [header.text for header in soup.find_all("tr")[1].find_all("th")]

#Create Data Frame
contract_data = pd.DataFrame(db_data[2:], columns = contract_header[1:])

#Clean Data
contract_data.dropna(how="all", axis=0, inplace=True)

contract_data.loc[:,'2018-19':'2023-24']=contract_data.loc[:,'2018-19':'2023-24'].replace("\$","",regex=True)
contract_data.loc[:,'2018-19':'2023-24']=contract_data.loc[:,'2018-19':'2023-24'].replace(",","",regex=True).apply(pd.to_numeric)

contract_data.loc[:,"Guaranteed"] = contract_data.loc[:,"Guaranteed"].replace("\$","", regex=True)
contract_data.loc[:,"Guaranteed"] = contract_data.loc[:,"Guaranteed"].replace(",","", regex=True).apply(pd.to_numeric)


contract_data.head()

,Player,Tm,2018-19,2019-20,2020-21,2021-22,2022-23,2023-24,Signed Using,Guaranteed
0,Stephen Curry,GSW,3.74572e+07,4.02318e+07,4.30064e+07,4.5781e+07,NaN,NaN,Bird Rights,166476240.0
1,Chris Paul,HOU,3.56542e+07,3.85065e+07,4.13588e+07,4.42111e+07,NaN,NaN,,159730592.0
2,LeBron James,LAL,3.56542e+07,3.74369e+07,3.92196e+07,4.10023e+07,NaN,NaN,,113310573.0
3,Russell Westbrook,OKC,3.535e+07,3.8178e+07,4.1006e+07,4.3848e+07,4.6662e+07,NaN,Bird Rights,158382000.0
4,Blake Griffin,DET,3.18739e+07,3.4235e+07,3.6596e+07,3.8957e+07,NaN,NaN,Bird Rights,102704892.0


In [122]:
from sqlalchemy import create_engine
import sqlite3 as sq

In [146]:
engine = create_engine(r"sqlite:///C:\Users\Lopezped\Documents\GitHub\NBA-Database-Creator\NBA_data.db")

data.to_sql("Player Totals", con = engine, if_exists= "replace", index= data["Player"])
contract_data.to_sql("Player Contracts", con = engine, if_exists="replace", index = contract_data["Player"])

In [149]:
connection = sq.connect("NBA_data.db")
c = connection.cursor()
c.execute("SELECT * FROM 'Player Contracts'")

In [150]:
print(c.fetchmany(5))

[('Stephen Curry', 'GSW', 37457154.0, 40231758.0, 43006362.0, 45780966.0, None, None, 'Bird Rights', 166476240.0), ('Chris Paul', 'HOU', 35654150.0, 38506482.0, 41358814.0, 44211146.0, None, None, '', 159730592.0), ('LeBron James', 'LAL', 35654150.0, 37436858.0, 39219565.0, 41002273.0, None, None, '', 113310573.0), ('Russell Westbrook', 'OKC', 35350000.0, 38178000.0, 41006000.0, 43848000.0, 46662000.0, None, 'Bird Rights', 158382000.0), ('Blake Griffin', 'DET', 31873932.0, 34234964.0, 36595996.0, 38957028.0, None, None, 'Bird Rights', 102704892.0)]


In [151]:
connection.close()